# LSTM 구현 예제, data 차원 이해

- lstm 구조적 이해와 lstm 적합하는 data 차원에 대한 이해에 집중

---

# import

In [81]:
from datasetsforecast.m4 import M4, M4Evaluation
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import time
from keras.layers import LSTM ,Dense, Bidirectional, Input, TimeDistributed
from keras.models import Sequential
import keras.backend as K
from keras.callbacks import EarlyStopping

---

# many to one data
- 입력값 : 데이터 시퀀스
- 출력값 : 단일 값
- toydata로 만들어보자

In [320]:
np.random.seed(1)
toydata = np.random.normal(1,1,50)

`-` input data
- input dim 를 5로 설정

In [224]:
X1 = toydata.reshape(-1,5,1)
X1.shape

(10, 5, 1)

`-` y
- 각 시퀀스의 평균 값으로 가정

In [225]:
y = X.mean(axis=1)
y = y.reshape(-1,1)
y.shape

(10, 1)

---

# many to many data
- 마찬가지로 input dim 5로 설정
- X, y가 동일한 차원

`-` input data
- input data 위 데이터와 크게 달라지지 않음

In [238]:
X1.shape

(10, 5, 1)

`-` y
- many to many이기 때문에 input dim과 동일한 시퀀스를 반환해야함
- 단순히 각 x에 평균을 뺀 값이라고 설정해보자

In [237]:
y2 = np.array([X1[i] - y1[i] for i in range(len(y1))])
y2.shape

(10, 5, 1)

---

# LSTM 모형적합

`-` 공통적용
- stacked lstm model로 2개의 LSTM layer
- return_sequences=True 1층에서 2층으로 모든 스텝의 출력이 전달되어야 하기 때문에
- cell을 1층, 2층 모두 64개로 적합(양방향은 2배가 된다)
- inputdim = 5

In [242]:
inputdim = X.shape[1]
inputdim

5

---

# seq to one, 단방향 모델

`-` 모델생성

In [279]:
K.clear_session()
model1 = Sequential() # Sequeatial Model
model1.add(LSTM(64,input_shape=(inputdim,1),return_sequences=True)) # 1층 단방향
model1.add(LSTM(64)) # 2층 단방향
model1.add(Dense(1))
model1.compile(loss='mean_squared_error', optimizer='adam')

In [280]:
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 5, 64)             16896     
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 49985 (195.25 KB)
Trainable params: 49985 (195.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


`-` 모델적합

In [293]:
np.random.seed(1)
early_stop = EarlyStopping(monitor='loss', patience=3, verbose=1)

model1.fit(X1, y1, epochs=100,
          batch_size=128, verbose=1, callbacks=[early_stop])


Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 1.0807
Epoch 2/100
1/1 [==============================] - 0s 6ms/step - loss: 1.0035
Epoch 3/100
1/1 [==============================] - 0s 6ms/step - loss: 0.9291
Epoch 4/100
1/1 [==============================] - 0s 5ms/step - loss: 0.8562
Epoch 5/100
1/1 [==============================] - 0s 5ms/step - loss: 0.7841
Epoch 6/100
1/1 [==============================] - 0s 5ms/step - loss: 0.7122
Epoch 7/100
1/1 [==============================] - 0s 5ms/step - loss: 0.6401
Epoch 8/100
1/1 [==============================] - 0s 5ms/step - loss: 0.5675
Epoch 9/100
1/1 [==============================] - 0s 5ms/step - loss: 0.4945
Epoch 10/100
1/1 [==============================] - 0s 5ms/step - loss: 0.4211
Epoch 11/100
1/1 [==============================] - 0s 6ms/step - loss: 0.3480
Epoch 12/100
1/1 [==============================] - 0s 5ms/step - loss: 0.2760
Epoch 13/100
1/1 [==============================] - 0s 5ms/ste

---

# seq to one, 양방향 모델
- Bidirectional
- build

`-` 모델생성

In [276]:
K.clear_session()
model2 = Sequential() # Sequeatial Model
model2.add(Bidirectional(LSTM(64,input_shape=(inputdim,1),return_sequences=True)) )
model2.add(Bidirectional(LSTM(64))) 
model2.add(Dense(1))
model2.build(input_shape=(None, inputdim, 1))
model2.compile(loss='mean_squared_error', optimizer='adam')

In [277]:
model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirection  (None, 5, 128)            33792     
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 128)               98816     
 onal)                                                           
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 132737 (518.50 KB)
Trainable params: 132737 (518.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


`-` 모델적합

In [294]:
np.random.seed(1)
early_stop = EarlyStopping(monitor='loss', patience=3, verbose=1)

model2.fit(X1, y1, epochs=100,
          batch_size=128, verbose=1, callbacks=[early_stop])


Epoch 1/100
1/1 [==============================] - 4s 4s/step - loss: 1.0356
Epoch 2/100
1/1 [==============================] - 0s 6ms/step - loss: 0.8413
Epoch 3/100
1/1 [==============================] - 0s 6ms/step - loss: 0.6649
Epoch 4/100
1/1 [==============================] - 0s 5ms/step - loss: 0.5047
Epoch 5/100
1/1 [==============================] - 0s 6ms/step - loss: 0.3604
Epoch 6/100
1/1 [==============================] - 0s 6ms/step - loss: 0.2335
Epoch 7/100
1/1 [==============================] - 0s 5ms/step - loss: 0.1280
Epoch 8/100
1/1 [==============================] - 0s 5ms/step - loss: 0.0501
Epoch 9/100
1/1 [==============================] - 0s 6ms/step - loss: 0.0079
Epoch 10/100
1/1 [==============================] - 0s 5ms/step - loss: 0.0081
Epoch 11/100
1/1 [==============================] - 0s 6ms/step - loss: 0.0457
Epoch 12/100
1/1 [==============================] - 0s 6ms/step - loss: 0.0921
Epoch 12: early stopping


---

# seq to many, 단방향 모델
- TimeDistributed
- 다음 층도 return sequences를 true로 지정

`-` 모델생성

In [322]:
K.clear_session()
model3 = Sequential() # Sequeatial Model
model3.add(LSTM(64,input_shape=(inputdim,1),return_sequences=True))# 1층 단방향
model3.add(LSTM(64,return_sequences=True))# 2층 또한 true로 지정
model3.add(TimeDistributed(Dense(1)))
#model3.build(input_shape=(None, inputdim, 1))
model3.compile(loss='mean_squared_error', optimizer='adam')

In [323]:
model3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 5, 64)             16896     
                                                                 
 lstm_1 (LSTM)               (None, 5, 64)             33024     
                                                                 
 time_distributed (TimeDist  (None, 5, 1)              65        
 ributed)                                                        
                                                                 
Total params: 49985 (195.25 KB)
Trainable params: 49985 (195.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


`-` 모델적합

In [295]:
np.random.seed(1)
early_stop = EarlyStopping(monitor='loss', patience=3, verbose=1)

model3.fit(X1, y2, epochs=100,
          batch_size=128, verbose=1, callbacks=[early_stop])


Epoch 1/100
1/1 [==============================] - 3s 3s/step - loss: 0.8379
Epoch 2/100
1/1 [==============================] - 0s 6ms/step - loss: 0.8274
Epoch 3/100
1/1 [==============================] - 0s 5ms/step - loss: 0.8186
Epoch 4/100
1/1 [==============================] - 0s 5ms/step - loss: 0.8113
Epoch 5/100
1/1 [==============================] - 0s 5ms/step - loss: 0.8054
Epoch 6/100
1/1 [==============================] - 0s 5ms/step - loss: 0.8007
Epoch 7/100
1/1 [==============================] - 0s 6ms/step - loss: 0.7970
Epoch 8/100
1/1 [==============================] - 0s 5ms/step - loss: 0.7945
Epoch 9/100
1/1 [==============================] - 0s 5ms/step - loss: 0.7927
Epoch 10/100
1/1 [==============================] - 0s 6ms/step - loss: 0.7916
Epoch 11/100
1/1 [==============================] - 0s 5ms/step - loss: 0.7906
Epoch 12/100
1/1 [==============================] - 0s 5ms/step - loss: 0.7895
Epoch 13/100
1/1 [==============================] - 0s 5ms/ste

---

# seq to many, 양방향 모델
- Bidirectional
- 2층 return_sequences=True
- TimeDistributed

`-` 모델생성

In [290]:
K.clear_session()
model4 = Sequential() # Sequeatial Model
model4.add(Bidirectional(LSTM(64,input_shape=(None,inputdim,1),return_sequences=True)) )# 1층 양방향
model4.add(Bidirectional(LSTM(64,return_sequences=True))) # 2층 양방향
model4.add(TimeDistributed(Dense(1)))
model4.build(input_shape=(None, inputdim, 1))
model4.compile(loss='mean_squared_error', optimizer='adam')

In [291]:
model4.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirection  (None, 5, 128)            33792     
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 5, 128)            98816     
 onal)                                                           
                                                                 
 time_distributed (TimeDist  (None, 5, 1)              129       
 ributed)                                                        
                                                                 
Total params: 132737 (518.50 KB)
Trainable params: 132737 (518.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


`-` 모델적합

In [296]:
np.random.seed(1)
early_stop = EarlyStopping(monitor='loss', patience=3, verbose=1)

model4.fit(X1, y2, epochs=100,
          batch_size=128, verbose=1, callbacks=[early_stop])


Epoch 1/100
1/1 [==============================] - 4s 4s/step - loss: 0.8112
Epoch 2/100
1/1 [==============================] - 0s 6ms/step - loss: 0.7942
Epoch 3/100
1/1 [==============================] - 0s 6ms/step - loss: 0.7783
Epoch 4/100
1/1 [==============================] - 0s 6ms/step - loss: 0.7634
Epoch 5/100
1/1 [==============================] - 0s 5ms/step - loss: 0.7492
Epoch 6/100
1/1 [==============================] - 0s 6ms/step - loss: 0.7359
Epoch 7/100
1/1 [==============================] - 0s 5ms/step - loss: 0.7233
Epoch 8/100
1/1 [==============================] - 0s 6ms/step - loss: 0.7114
Epoch 9/100
1/1 [==============================] - 0s 6ms/step - loss: 0.7002
Epoch 10/100
1/1 [==============================] - 0s 5ms/step - loss: 0.6899
Epoch 11/100
1/1 [==============================] - 0s 6ms/step - loss: 0.6803
Epoch 12/100
1/1 [==============================] - 0s 6ms/step - loss: 0.6717
Epoch 13/100
1/1 [==============================] - 0s 6ms/ste

---

# TEST

In [301]:
pred1 = model1.predict(X1)
pred2 = model2.predict(X1)
pred3 = model3.predict(X1)
pred4 = model4.predict(X1)

1/1 [==============================] - 0s 16ms/step


In [314]:
pred1.shape,pred2.shape,pred3.shape,pred4.shape

((10, 1), (10, 1), (10, 5, 1), (10, 5, 1))

In [310]:
from sklearn.metrics import mean_squared_error

print(mean_squared_error(y1,pred1))
print(mean_squared_error(y1,pred2))
print(mean_squared_error(y2.flatten(),pred3.flatten()))
print(mean_squared_error(y2.flatten(),pred4.flatten()))

0.10189318358518569
0.11652249485240232
0.08902064067539099
0.05776927625836931
